In [ ]:
# imports
import numpy as np
import pandas as pd
import re

In [ ]:
# read data from csv files
all_essays = pd.read_csv('./data/train.csv', index_col=0)
all_vocab_words = pd.read_csv('./data/unigram_freq.csv', index_col=0)

In [ ]:
# building dictionary with vocab word as key and count as value
vocab_dict = {}

for data in all_vocab_words.iterrows():
  vocab_dict[data[0]] = data[1][0]

In [ ]:
cleaned_essays = []
misspelled = []

for essay in enumerate(all_essays):
  essay_wo_punc = re.sub(r'[^\w\s]', '', essay)
  essay_lower = essay_wo_punc.lower()
  split_essay = re.split('[^a-zA-Z]+', essay_lower)
  cleaned_essays.append(split_essay)

  misspell_count = 0
  for word in split_essay:
    if word not in vocab_dict and word != '':
      misspell_count += 1
  misspelled.append(misspell_count)